In [49]:
pip install mlflow

In [50]:
import os


import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import boto3
import mlflow
from mlflow.models import infer_signature

In [51]:
mlflow.set_tracking_uri("http://127.0.0.1:8001")


In [52]:
mlflow.create_experiment(name="change_mlflow_artifact_registry")
mlflow.set_experiment("change_mlflow_artifact_registry")
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

In [53]:

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)


c:\Users\ual-laptop\Documents\Cloud-Assgn10\Dev\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [54]:
# Predict on the test set
y_pred = lr.predict(X_test)
print(y_pred)


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [55]:

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)
# Start an MLflow run
print(accuracy)

1.0


In [56]:
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris-model",
        signature=signature,
        input_example=X_train,
        registered_model_name="basic_lr_iris_model",
    )



Successfully registered model 'basic_lr_iris_model'.
2024/12/08 23:45:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: basic_lr_iris_model, version 1


🏃 View run tasteful-snail-238 at: http://127.0.0.1:8001/#/experiments/413214295984941141/runs/6e790be41db048c7b54296439397c6dc
🧪 View experiment at: http://127.0.0.1:8001/#/experiments/413214295984941141


Created version '1' of model 'basic_lr_iris_model'.


In [57]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
